## Imports

In [1]:
import os
import torch
import torch.nn as nn
import wandb
import numpy as np
import torchvision.transforms

from mmcv_csn import ResNet3dCSN
from csn import csn50
from i3d_head import I3DHead
from autoencoder import EncoderDecoder
from depth_head import DepthHead
from scheduler import GradualWarmupScheduler
from mmaction.datasets import build_dataset

os.chdir('../')

/home/sadat/miniconda3/envs/dataloader/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/sadat/miniconda3/envs/dataloader/lib/python3.8/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


In [2]:
wandb.init(entity="cares", project="autoencoder",
           group="wlasl-10", name="depth")

wandb: Currently logged in as: sttaseen (cares). Use `wandb login --relogin` to force relogin


## Device Agnostic Code

In [3]:
# Set up device agnostic code
try:
    device = 'mps' if torch.backends.mps.is_available() else 'cpu'
except:
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

## Dataset

In [4]:
train_cfg=dict(
    type='RawframeDataset',
    ann_file='data/wlasl10/train_annotations.txt',
    data_prefix='data/wlasl10/rawframes',
    pipeline=[
        dict(
            type='SampleFrames',
            clip_len=32,
            frame_interval=2,
            num_clips=1),
        dict(type='RawFrameDecode'),
        dict(type='Resize', scale=(-1, 256)),
        dict(type='RandomResizedCrop'),
        dict(type='Resize', scale=(224, 224), keep_ratio=False),
        dict(type='Flip', flip_ratio=0.5),
        dict(
            type='Normalize',
            mean=[123.675, 116.28, 103.53],
            std=[58.395, 57.12, 57.375],
            to_bgr=False),
        dict(type='FormatShape', input_format='NCTHW'),
        dict(type='Collect', keys=['imgs', 'label'], meta_keys=[]),
        dict(type='ToTensor', keys=['imgs', 'label'])
    ])


test_cfg=dict(
        type='RawframeDataset',
        ann_file='data/wlasl10/test_annotations.txt',
        data_prefix='data/wlasl10/rawframes',
        pipeline=[
            dict(
                type='SampleFrames',
                clip_len=32,
                frame_interval=2,
                num_clips=1,
                test_mode=True),
            dict(type='RawFrameDecode'),
            dict(type='Resize', scale=(-1, 256)),
            dict(type='CenterCrop', crop_size=224),
            dict(
                type='Normalize',
                mean=[123.675, 116.28, 103.53],
                std=[58.395, 57.12, 57.375],
                to_bgr=False),
            dict(type='FormatShape', input_format='NCTHW'),
            dict(type='Collect', keys=['imgs', 'label'], meta_keys=[]),
            dict(type='ToTensor', keys=['imgs'])
    ])

In [5]:
work_dir = 'work_dirs/wlasl10-depth-dataset/'

os.makedirs(work_dir, exist_ok=True)

In [6]:
# Building the datasets

batch_size = 2

train_dataset = build_dataset(train_cfg)
test_dataset = build_dataset(test_cfg)

# Setting up dataloaders
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                    batch_size=batch_size,
                                    shuffle=True,
                                    num_workers=4,
                                    pin_memory=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                    batch_size=1,
                                    shuffle=True,
                                    num_workers=4,
                                    pin_memory=True)

## Model

In [7]:
# Create a CSN model
encoder = ResNet3dCSN(
    pretrained2d=False,
    # pretrained=None,
    pretrained='https://download.openmmlab.com/mmaction/recognition/csn/ircsn_from_scratch_r50_ig65m_20210617-ce545a37.pth',
    depth=50,
    with_pool2=False,
    bottleneck_mode='ir',
    norm_eval=True,
    zero_init_residual=False,
    bn_frozen=True
)

encoder.init_weights()

depth_head = DepthHead()

decoder = I3DHead(num_classes=400,
                 in_channels=2048,
                 spatial_type='avg',
                 dropout_ratio=0.5,
                 init_std=0.01)

decoder.init_weights()

model = EncoderDecoder(encoder, decoder, depth_head)

### Setup MiDaS

In [8]:
# Set up MiDaS depth model
model_type = "DPT_Large"     # MiDaS v3 - Large     (highest accuracy, slowest inference speed)
# model_type = "DPT_Hybrid"   # MiDaS v3 - Hybrid    (medium accuracy, medium inference speed)
# model_type = "MiDaS_small"  # MiDaS v2.1 - Small   (lowest accuracy, highest inference speed)

midas = torch.hub.load("intel-isl/MiDaS", model_type)
midas.to(device)
midas.eval()

Using cache found in /home/sadat/.cache/torch/hub/intel-isl_MiDaS_master


DPTDepthModel(
  (pretrained): Module(
    (model): VisionTransformer(
      (patch_embed): PatchEmbed(
        (proj): Conv2d(3, 1024, kernel_size=(16, 16), stride=(16, 16))
        (norm): Identity()
      )
      (pos_drop): Dropout(p=0.0, inplace=False)
      (norm_pre): Identity()
      (blocks): Sequential(
        (0): Block(
          (norm1): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
          (attn): Attention(
            (qkv): Linear(in_features=1024, out_features=3072, bias=True)
            (attn_drop): Dropout(p=0.0, inplace=False)
            (proj): Linear(in_features=1024, out_features=1024, bias=True)
            (proj_drop): Dropout(p=0.0, inplace=False)
          )
          (ls1): Identity()
          (drop_path1): Identity()
          (norm2): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
          (mlp): Mlp(
            (fc1): Linear(in_features=1024, out_features=4096, bias=True)
            (act): GELU()
            (drop1): Dropout(p=0

In [9]:
def estimate_depth(images):
    with torch.no_grad():
        depth = midas(images.permute(0,2,1,3,4).reshape(-1,3,224,224))

        depth = torch.nn.functional.interpolate(
            depth.unsqueeze(1),
            size=(224,224),
            mode="bicubic",
            align_corners=False,
        ).squeeze()
        
    return depth.reshape(-1, 1, 32, 224, 224)

## Optimizer

In [10]:
# Specify optimizer
optimizer = torch.optim.SGD(
    model.parameters(), lr=0.000125, momentum=0.9, weight_decay=0.00001)

# Specify Loss
loss_cls = nn.CrossEntropyLoss()
loss_depth = nn.MSELoss()

# Specify total epochs
epochs = 100

# Specify learning rate scheduler
lr_scheduler = torch.optim.lr_scheduler.StepLR(
    optimizer, step_size=120, gamma=0.1)

scheduler_steplr = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[34, 84], gamma=0.1)
scheduler = GradualWarmupScheduler(optimizer, multiplier=1, total_epoch=16, after_scheduler=scheduler_steplr)

model.to(device)

EncoderDecoder(
  (encoder): ResNet3dCSN(
    (conv1): ConvModule(
      (conv): Conv3d(3, 64, kernel_size=(3, 7, 7), stride=(1, 2, 2), padding=(1, 3, 3), bias=False)
      (bn): BatchNorm3d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (activate): ReLU(inplace=True)
    )
    (maxpool): MaxPool3d(kernel_size=(1, 3, 3), stride=(1, 2, 2), padding=(0, 1, 1), dilation=1, ceil_mode=False)
    (pool2): MaxPool3d(kernel_size=(2, 1, 1), stride=(2, 1, 1), padding=0, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): CSNBottleneck3d(
        (conv1): ConvModule(
          (conv): Conv3d(64, 64, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
          (bn): BatchNorm3d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
          (activate): ReLU(inplace=True)
        )
        (conv2): Sequential(
          (0): ConvModule(
            (conv): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), groups=64, bi

In [11]:
# x0=torch.rand((1,64,32,112,112)).to(device)
# x1=torch.rand((1,256,32,56,56)).to(device)
# x2=torch.rand((1,512,16,28,28)).to(device)
# x3=torch.rand((1,1024,8,14,14)).to(device)
# x4=torch.rand((1,2048,4,7,7)).to(device)
# x = (x0,x1,x2,x3,x4)

In [12]:
# depth_head(x).squeeze().shape

## Train Loop

In [13]:
# Setup wandb
# wandb.watch(model, log_freq=10)

def top_k_accuracy(scores, labels, topk=(1, )):
    """Calculate top k accuracy score.
    Args:
        scores (list[np.ndarray]): Prediction scores for each class.
        labels (list[int]): Ground truth labels.
        topk (tuple[int]): K value for top_k_accuracy. Default: (1, ).
    Returns:
        list[float]: Top k accuracy score for each k.
    """
    res = np.zeros(len(topk))
    labels = np.array(labels)[:, np.newaxis]
    for i, k in enumerate(topk):
        max_k_preds = np.argsort(scores, axis=1)[:, -k:][:, ::-1]
        match_array = np.logical_or.reduce(max_k_preds == labels, axis=1)
        topk_acc_score = match_array.sum() / match_array.shape[0]
        res[i] = topk_acc_score

    return res


def train_one_epoch(epoch_index, interval=5):
    """Run one epoch for training.
    Args:
        epoch_index (int): Current epoch.
        interval (int): Frequency at which to print logs.
    Returns:
        last_loss (float): Loss value for the last batch.
    """
    running_loss = 0.
    last_loss = 0.

    # Here, we use enumerate(training_loader) instead of
    # iter(training_loader) so that we can track the batch
    # index and do some intra-epoch reporting
    for i, x in enumerate(train_loader):
        # Every data instance is an input + label pair
        images, targets = x['imgs'].to(device), x['label'].to(device)
        images = images.reshape((-1, ) + images.shape[2:])
        targets = targets.reshape(-1, )
        
        # Zero your gradients for every batch!
        optimizer.zero_grad()

        # Make predictions for this batch
        cls_score, predicted_depth = model(images)
        
        # Estimate depth using MiDaS
        depth = estimate_depth(images)
        
        # Get losses
        loss_cls_score = loss_cls(cls_score, targets)
        loss_depth_score = loss_depth(predicted_depth, depth)
        loss = 0.8 * loss_cls_score + 0.2 * loss_depth_score
        
        # Compute the loss and its gradients
        loss.backward()

        # Gradient Clipping
        torch.nn.utils.clip_grad_norm_(
            model.parameters(), max_norm=40, norm_type=2.0)

        # Adjust learning weights
        optimizer.step()
        

        # Gather data and report
        running_loss += loss.item()
        if i % interval == interval-1:
            last_loss = running_loss / interval  # loss per batch
            print(
                f'Epoch [{epoch_index}][{i+1}/{len(train_loader)}], loss_cls: {loss_cls_score.item():.5}, depth_loss: {loss_depth_score.item():.5} lr: {scheduler.get_last_lr()[0]:.5e}, loss: {last_loss:.5}')
            running_loss = 0.

    return last_loss, scheduler.get_last_lr()[0]


def validate():
    """Run one epoch for validation.
    Returns:
        avg_vloss (float): Validation loss value for the last batch.
        top1_acc (float): Top-1 accuracy in decimal.
        top5_acc (float): Top-5 accuracy in decimal.
    """
    running_vloss = 0.0
    running_vacc = np.zeros(2)

    print('Evaluating top_k_accuracy...')

    with torch.inference_mode():
        for i, x in enumerate(test_loader):
            vimages, vtargets = x['imgs'].to(device), x['label'].to(device)
            vimages = vimages.reshape((-1, ) + vimages.shape[2:])
            vtargets = vtargets.reshape(-1, )
            
            # Make predictions for this batch
            cls_score, predicted_depth = model(vimages)

            # Estimate depth using MiDaS
            depth = estimate_depth(vimages)

            # Get losses
            loss_cls_score = loss_cls(cls_score, vtargets)
            loss_depth_score = loss_depth(predicted_depth, depth)
            vloss = 0.8 * loss_cls_score + 0.2 * loss_depth_score
            
            running_vloss += vloss

            running_vacc += top_k_accuracy(cls_score.detach().cpu().numpy(),
                                           vtargets.detach().cpu().numpy(), topk=(1, 5))

    avg_vloss = running_vloss / (i + 1)

    acc = running_vacc/len(test_loader)
    top1_acc = acc[0].item()
    top5_acc = acc[1].item()

    return (avg_vloss, top1_acc, top5_acc)


# Train Loop
best_vloss = 1_000_000.

for epoch in range(epochs):
    # Turn on gradient tracking and do a forward pass
    model.train(True)
    avg_loss, learning_rate = train_one_epoch(epoch+1)

    # Turn off  gradients for reporting
    model.train(False)

    avg_vloss, top1_acc, top5_acc = validate()

    print(
        f'top1_acc: {top1_acc:.4}, top5_acc: {top5_acc:.4}, train_loss: {avg_loss:.5}, val_loss: {avg_vloss:.5}')

    # Track best performance, and save the model's state
    if avg_vloss < best_vloss:
        best_vloss = avg_vloss
        model_path = work_dir + f'epoch_{epoch+1}.pth'
        print(f'Saving checkpoint at {epoch+1} epochs...')
        torch.save(model.state_dict(), model_path)

     # Adjust learning rate
    scheduler.step()

    # Track wandb
    wandb.log({'train/loss': avg_loss,
               'train/learning_rate': learning_rate,
               'val/loss': avg_vloss,
               'val/top1_accuracy': top1_acc,
               'val/top5_accuracy': top5_acc})

Epoch [1][5/100], loss_cls: 5.957, depth_loss: 185.78 lr: 0.00000e+00, loss: 61.723
Epoch [1][10/100], loss_cls: 6.0782, depth_loss: 247.7 lr: 0.00000e+00, loss: 49.425
Epoch [1][15/100], loss_cls: 6.0302, depth_loss: 149.94 lr: 0.00000e+00, loss: 60.489
Epoch [1][20/100], loss_cls: 5.8484, depth_loss: 188.43 lr: 0.00000e+00, loss: 53.803
Epoch [1][25/100], loss_cls: 6.1516, depth_loss: 315.83 lr: 0.00000e+00, loss: 65.932
Epoch [1][30/100], loss_cls: 6.1372, depth_loss: 263.03 lr: 0.00000e+00, loss: 64.128
Epoch [1][35/100], loss_cls: 6.1178, depth_loss: 199.56 lr: 0.00000e+00, loss: 59.029
Epoch [1][40/100], loss_cls: 5.7811, depth_loss: 298.41 lr: 0.00000e+00, loss: 63.43
Epoch [1][45/100], loss_cls: 5.9172, depth_loss: 311.47 lr: 0.00000e+00, loss: 70.877
Epoch [1][50/100], loss_cls: 5.9634, depth_loss: 166.38 lr: 0.00000e+00, loss: 49.024
Epoch [1][55/100], loss_cls: 5.9322, depth_loss: 218.07 lr: 0.00000e+00, loss: 62.312
Epoch [1][60/100], loss_cls: 6.1725, depth_loss: 334.26 lr

Epoch [5][55/100], loss_cls: 3.2985, depth_loss: 121.6 lr: 3.12500e-05, loss: 23.588
Epoch [5][60/100], loss_cls: 3.2606, depth_loss: 101.36 lr: 3.12500e-05, loss: 21.682
Epoch [5][65/100], loss_cls: 2.7699, depth_loss: 64.207 lr: 3.12500e-05, loss: 21.874
Epoch [5][70/100], loss_cls: 2.3693, depth_loss: 87.796 lr: 3.12500e-05, loss: 19.021
Epoch [5][75/100], loss_cls: 2.1037, depth_loss: 49.692 lr: 3.12500e-05, loss: 15.965
Epoch [5][80/100], loss_cls: 2.4505, depth_loss: 93.658 lr: 3.12500e-05, loss: 20.284
Epoch [5][85/100], loss_cls: 4.1476, depth_loss: 44.358 lr: 3.12500e-05, loss: 20.281
Epoch [5][90/100], loss_cls: 2.4698, depth_loss: 34.601 lr: 3.12500e-05, loss: 12.501
Epoch [5][95/100], loss_cls: 3.0408, depth_loss: 66.134 lr: 3.12500e-05, loss: 17.334
Epoch [5][100/100], loss_cls: 1.7856, depth_loss: 46.679 lr: 3.12500e-05, loss: 14.633
Evaluating top_k_accuracy...
top1_acc: 0.1143, top5_acc: 0.6286, train_loss: 14.633, val_loss: 11.365
Saving checkpoint at 5 epochs...
Epoch

top1_acc: 0.1429, top5_acc: 0.4857, train_loss: 6.8287, val_loss: 5.8968
Saving checkpoint at 9 epochs...
Epoch [10][5/100], loss_cls: 1.5058, depth_loss: 24.909 lr: 7.03125e-05, loss: 5.9367
Epoch [10][10/100], loss_cls: 1.5175, depth_loss: 29.17 lr: 7.03125e-05, loss: 6.1134
Epoch [10][15/100], loss_cls: 2.2007, depth_loss: 33.926 lr: 7.03125e-05, loss: 7.8609
Epoch [10][20/100], loss_cls: 2.9843, depth_loss: 15.989 lr: 7.03125e-05, loss: 7.2094
Epoch [10][25/100], loss_cls: 2.7221, depth_loss: 39.553 lr: 7.03125e-05, loss: 6.7311
Epoch [10][30/100], loss_cls: 2.4088, depth_loss: 17.656 lr: 7.03125e-05, loss: 5.8487
Epoch [10][35/100], loss_cls: 3.2663, depth_loss: 26.673 lr: 7.03125e-05, loss: 8.4801
Epoch [10][40/100], loss_cls: 1.906, depth_loss: 24.622 lr: 7.03125e-05, loss: 6.7431
Epoch [10][45/100], loss_cls: 2.8371, depth_loss: 23.992 lr: 7.03125e-05, loss: 6.9391
Epoch [10][50/100], loss_cls: 0.71914, depth_loss: 23.455 lr: 7.03125e-05, loss: 6.2521
Epoch [10][55/100], loss_c

Epoch [14][45/100], loss_cls: 1.6939, depth_loss: 11.77 lr: 1.01563e-04, loss: 6.3929
Epoch [14][50/100], loss_cls: 3.3997, depth_loss: 31.633 lr: 1.01563e-04, loss: 6.9439
Epoch [14][55/100], loss_cls: 2.132, depth_loss: 14.638 lr: 1.01563e-04, loss: 6.5775
Epoch [14][60/100], loss_cls: 1.5259, depth_loss: 55.927 lr: 1.01563e-04, loss: 7.6575
Epoch [14][65/100], loss_cls: 0.0015866, depth_loss: 19.757 lr: 1.01563e-04, loss: 5.9938
Epoch [14][70/100], loss_cls: 4.3467, depth_loss: 11.743 lr: 1.01563e-04, loss: 5.0883
Epoch [14][75/100], loss_cls: 4.1739, depth_loss: 26.133 lr: 1.01563e-04, loss: 8.1704
Epoch [14][80/100], loss_cls: 1.5411, depth_loss: 41.423 lr: 1.01563e-04, loss: 5.7956
Epoch [14][85/100], loss_cls: 2.4287, depth_loss: 43.748 lr: 1.01563e-04, loss: 9.3122
Epoch [14][90/100], loss_cls: 1.0251, depth_loss: 27.085 lr: 1.01563e-04, loss: 5.0756
Epoch [14][95/100], loss_cls: 5.7041, depth_loss: 34.195 lr: 1.01563e-04, loss: 7.5132
Epoch [14][100/100], loss_cls: 1.5301, dep

Epoch [18][90/100], loss_cls: 4.7135, depth_loss: 31.027 lr: 1.25000e-04, loss: 7.9469
Epoch [18][95/100], loss_cls: 2.9992, depth_loss: 18.535 lr: 1.25000e-04, loss: 5.8633
Epoch [18][100/100], loss_cls: 3.1691, depth_loss: 41.909 lr: 1.25000e-04, loss: 7.4427
Evaluating top_k_accuracy...
top1_acc: 0.3714, top5_acc: 0.9143, train_loss: 7.4427, val_loss: 4.7797
Epoch [19][5/100], loss_cls: 3.8984, depth_loss: 19.905 lr: 1.25000e-04, loss: 6.2731
Epoch [19][10/100], loss_cls: 2.4005, depth_loss: 17.954 lr: 1.25000e-04, loss: 4.8124
Epoch [19][15/100], loss_cls: 0.0080468, depth_loss: 36.803 lr: 1.25000e-04, loss: 4.9173
Epoch [19][20/100], loss_cls: 2.0369, depth_loss: 17.396 lr: 1.25000e-04, loss: 5.9393
Epoch [19][25/100], loss_cls: 1.3523, depth_loss: 27.371 lr: 1.25000e-04, loss: 4.1416
Epoch [19][30/100], loss_cls: 2.2661, depth_loss: 10.254 lr: 1.25000e-04, loss: 4.7529
Epoch [19][35/100], loss_cls: 2.0465, depth_loss: 67.635 lr: 1.25000e-04, loss: 7.8822
Epoch [19][40/100], loss_

...

Epoch [80][25/100], loss_cls: 4.0531e-06, depth_loss: 7.9365 lr: 1.25000e-05, loss: 1.9785
Epoch [80][30/100], loss_cls: 4.3511e-06, depth_loss: 8.5307 lr: 1.25000e-05, loss: 1.9959
Epoch [80][35/100], loss_cls: 0.13922, depth_loss: 7.8189 lr: 1.25000e-05, loss: 2.875
Epoch [80][40/100], loss_cls: 0.00016716, depth_loss: 9.1286 lr: 1.25000e-05, loss: 2.2409
Epoch [80][45/100], loss_cls: 1.2156, depth_loss: 13.384 lr: 1.25000e-05, loss: 3.1723
Epoch [80][50/100], loss_cls: 3.6287, depth_loss: 6.4857 lr: 1.25000e-05, loss: 2.5983
Epoch [80][55/100], loss_cls: 0.0021141, depth_loss: 14.328 lr: 1.25000e-05, loss: 3.0757
Epoch [80][60/100], loss_cls: 2.4438e-06, depth_loss: 7.0236 lr: 1.25000e-05, loss: 2.1253
Epoch [80][65/100], loss_cls: 0.00080022, depth_loss: 6.514 lr: 1.25000e-05, loss: 1.7312
Epoch [80][70/100], loss_cls: 0.0003093, depth_loss: 5.2744 lr: 1.25000e-05, loss: 2.2002
Epoch [80][75/100], loss_cls: 0.014956, depth_loss: 10.82 lr: 1.25000e-05, loss: 3.252
Epoch [80][80/100]

Epoch [84][60/100], loss_cls: 0.018004, depth_loss: 11.299 lr: 1.25000e-05, loss: 2.7194
Epoch [84][65/100], loss_cls: 0.45388, depth_loss: 5.9812 lr: 1.25000e-05, loss: 3.4728
Epoch [84][70/100], loss_cls: 0.00022879, depth_loss: 7.0064 lr: 1.25000e-05, loss: 2.8467
Epoch [84][75/100], loss_cls: 1.2981, depth_loss: 7.4675 lr: 1.25000e-05, loss: 2.5909
Epoch [84][80/100], loss_cls: 4.5715e-05, depth_loss: 11.367 lr: 1.25000e-05, loss: 2.8168
Epoch [84][85/100], loss_cls: 0.54589, depth_loss: 20.342 lr: 1.25000e-05, loss: 2.5406
Epoch [84][90/100], loss_cls: 0.0028644, depth_loss: 5.913 lr: 1.25000e-05, loss: 1.485
Epoch [84][95/100], loss_cls: 1.5001, depth_loss: 10.202 lr: 1.25000e-05, loss: 3.0482
Epoch [84][100/100], loss_cls: 4.0531e-06, depth_loss: 8.5726 lr: 1.25000e-05, loss: 2.0705
Evaluating top_k_accuracy...
top1_acc: 0.8286, top5_acc: 0.9714, train_loss: 2.0705, val_loss: 2.6109
Saving checkpoint at 84 epochs...
Epoch [85][5/100], loss_cls: 0.00014173, depth_loss: 5.5484 lr:

Epoch [88][95/100], loss_cls: 0.61735, depth_loss: 9.3129 lr: 1.25000e-05, loss: 2.4265
Epoch [88][100/100], loss_cls: 0.0017603, depth_loss: 15.478 lr: 1.25000e-05, loss: 3.0308
Evaluating top_k_accuracy...
top1_acc: 0.8286, top5_acc: 0.9714, train_loss: 3.0308, val_loss: 2.6973
Epoch [89][5/100], loss_cls: 0.0030994, depth_loss: 8.5538 lr: 1.25000e-05, loss: 1.9543
Epoch [89][10/100], loss_cls: 0.28669, depth_loss: 15.187 lr: 1.25000e-05, loss: 2.4544
Epoch [89][15/100], loss_cls: 0.76941, depth_loss: 16.075 lr: 1.25000e-05, loss: 1.9111
Epoch [89][20/100], loss_cls: 0.00013571, depth_loss: 9.0497 lr: 1.25000e-05, loss: 1.9338
Epoch [89][25/100], loss_cls: 7.8081e-06, depth_loss: 6.8521 lr: 1.25000e-05, loss: 1.8326
Epoch [89][30/100], loss_cls: 0.00067159, depth_loss: 15.901 lr: 1.25000e-05, loss: 2.2147
Epoch [89][35/100], loss_cls: 1.8835e-05, depth_loss: 10.204 lr: 1.25000e-05, loss: 3.4738
Epoch [89][40/100], loss_cls: 0.16756, depth_loss: 11.059 lr: 1.25000e-05, loss: 3.6777
Ep

Epoch [93][25/100], loss_cls: 0.00072974, depth_loss: 11.787 lr: 1.25000e-05, loss: 1.5308
Epoch [93][30/100], loss_cls: 4.1723e-07, depth_loss: 7.2311 lr: 1.25000e-05, loss: 2.8042
Epoch [93][35/100], loss_cls: 0.006242, depth_loss: 10.108 lr: 1.25000e-05, loss: 3.0967
Epoch [93][40/100], loss_cls: 5.4836e-06, depth_loss: 13.143 lr: 1.25000e-05, loss: 1.9388
Epoch [93][45/100], loss_cls: 0.0026677, depth_loss: 6.8645 lr: 1.25000e-05, loss: 2.3685
Epoch [93][50/100], loss_cls: 1.3485, depth_loss: 19.69 lr: 1.25000e-05, loss: 2.6409
Epoch [93][55/100], loss_cls: 0.016438, depth_loss: 16.952 lr: 1.25000e-05, loss: 2.2846
Epoch [93][60/100], loss_cls: 0.57794, depth_loss: 20.051 lr: 1.25000e-05, loss: 3.1067
Epoch [93][65/100], loss_cls: 0.0005427, depth_loss: 6.5175 lr: 1.25000e-05, loss: 1.6373
Epoch [93][70/100], loss_cls: 0.01983, depth_loss: 6.1187 lr: 1.25000e-05, loss: 1.7452
Epoch [93][75/100], loss_cls: 1.8358e-05, depth_loss: 6.8162 lr: 1.25000e-05, loss: 2.1225
Epoch [93][80/10

Epoch [97][60/100], loss_cls: 0.0011539, depth_loss: 18.441 lr: 1.25000e-05, loss: 2.8249
Epoch [97][65/100], loss_cls: 1.1412, depth_loss: 22.95 lr: 1.25000e-05, loss: 4.2271
Epoch [97][70/100], loss_cls: 0.006056, depth_loss: 7.6871 lr: 1.25000e-05, loss: 2.2675
Epoch [97][75/100], loss_cls: 0.00023127, depth_loss: 7.0918 lr: 1.25000e-05, loss: 2.0532
Epoch [97][80/100], loss_cls: 0.00011026, depth_loss: 9.387 lr: 1.25000e-05, loss: 2.2011
Epoch [97][85/100], loss_cls: 3.0577e-05, depth_loss: 5.7686 lr: 1.25000e-05, loss: 2.5085
Epoch [97][90/100], loss_cls: 4.4821e-05, depth_loss: 13.868 lr: 1.25000e-05, loss: 2.2358
Epoch [97][95/100], loss_cls: 2.7656e-05, depth_loss: 12.461 lr: 1.25000e-05, loss: 2.4304
Epoch [97][100/100], loss_cls: 0.71404, depth_loss: 19.784 lr: 1.25000e-05, loss: 1.995
Evaluating top_k_accuracy...
top1_acc: 0.8286, top5_acc: 0.9714, train_loss: 1.995, val_loss: 2.6627
Epoch [98][5/100], loss_cls: 0.013766, depth_loss: 8.0234 lr: 1.25000e-05, loss: 2.7161
Epoc